# Enhanced License Plate Detection with Bidirectional FPN

This notebook implements an improved license plate detection model using a bidirectional Feature Pyramid Network (BiFPN) architecture. The goal is to achieve a mean IoU of 0.6 or higher, specifically by improving detection of small license plates which form the majority of the dataset.

### Key improvements over the previous model:

1. **Enhanced BiFPN Architecture** with bidirectional feature flow and attention mechanisms
2. **Size-Aware Loss Functions** that place greater emphasis on width/height accuracy
3. **Small Plate Focused Augmentation** with adaptive strategies based on plate size
4. **Improved Training Strategies** including cyclic learning rates and focused early stopping

## Step 1: Imports and Setup

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
import sys
import random
import gc
import time
from IPython.display import clear_output

# Add the project root directory to the path
project_root = str(Path(os.getcwd()).parent)
if project_root not in sys.path:
    sys.path.append(project_root)

# Import custom modules from the license plate detection package
from license_plate_detection.data.enhanced_augmentation import (
    augment_data_with_small_plate_focus, 
    create_tf_data_augmenter
)
from license_plate_detection.models.enhanced_losses import (
    ultra_plate_detection_loss,
    enhanced_ciou_loss,
    enhanced_focal_loss,
    enhanced_iou_metric
)
from license_plate_detection.models.enhanced_fpn import (
    create_bifpn_license_plate_detector,
    enable_gradient_checkpointing
)
from license_plate_detection.train.optimization import (
    create_lr_scheduler,
    setup_gpu_memory_growth,
    limit_gpu_memory,
    clean_memory,
    optimize_memory_usage
)
from license_plate_detection.train.trainer  import create_efficient_data_pipeline
from license_plate_detection.evaluation.evaluator import evaluate_model_comprehensive
from license_plate_detection.evaluation.error_analysis import analyze_error_patterns
from license_plate_detection.utils.visualization import (
    visualize_bounding_boxes, 
    visualize_augmentation,
    plot_predictions
)

# Set seeds for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Set image size - using a higher resolution for better small plate detection
IMAGE_SIZE = (320, 320)  # Previous model used (224, 224)

# Configure TensorFlow to handle GPU memory more efficiently
setup_gpu_memory_growth()

# Check for available GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
for device in tf.config.list_physical_devices():
    print(f"Found device: {device.name}")

## Step 2: Data Loading and Preprocessing

In [ ]:
# Load dataset with increased image resolution
print("Loading and preprocessing dataset...")
df, X, y = load_dataset(
    dataset_path=os.path.join(project_root, 'Dataset'),
    image_size=IMAGE_SIZE,
    normalize=True,
    verbose=True
)

# Display dataset statistics
print(f"Dataset loaded: {len(X)} images with shape {X[0].shape}")

# Visualize a few examples
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i in range(min(6, len(X))):
    visualize_bounding_boxes(X[i], [y[i]], ax=axes[i])
    
plt.tight_layout()
plt.show()

### Analyze distribution of plate sizes

In [ ]:
# Calculate plate sizes (area) as percentage of image
plate_areas = np.array([box[2] * box[3] for box in y])  # width * height

# Define thresholds for small, medium, and large plates
small_threshold = 0.05  # 5% of image area
large_threshold = 0.1   # 10% of image area

# Count plates in each category
small_count = np.sum(plate_areas < small_threshold)
large_count = np.sum(plate_areas >= large_threshold)
medium_count = len(plate_areas) - small_count - large_count

print(f"Plate size distribution:")
print(f"  Small plates (<{small_threshold*100}% of image): {small_count} ({small_count/len(y)*100:.1f}%)")
print(f"  Medium plates: {medium_count} ({medium_count/len(y)*100:.1f}%)")
print(f"  Large plates (>={large_threshold*100}% of image): {large_count} ({large_count/len(y)*100:.1f}%)")

# Plot histogram of plate sizes
plt.figure(figsize=(10, 6))
plt.hist(plate_areas * 100, bins=30, alpha=0.7, color='blue')
plt.axvline(small_threshold * 100, color='r', linestyle='--', alpha=0.5, label=f'Small plate threshold ({small_threshold*100}%)')
plt.axvline(large_threshold * 100, color='g', linestyle='--', alpha=0.5, label=f'Large plate threshold ({large_threshold*100}%)')
plt.xlabel('Plate Area (% of image)')
plt.ylabel('Count')
plt.title('Distribution of License Plate Sizes')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

### Enhanced Data Augmentation

In [ ]:
# Apply the enhanced augmentation with focus on small plates
print("Generating augmented dataset with special focus on small plates...")
X_aug, y_aug = augment_data_with_small_plate_focus(
    X, y, 
    augmentation_factor=5  # Increased from 4 in the original
)

# Visualize some augmented samples
visualize_augmentation(X, y, X_aug, y_aug, num_samples=4)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = split_dataset(X_aug, y_aug, test_size=0.2, random_state=RANDOM_SEED)
print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")

## Step 3: Enhanced BiFPN Architecture Implementation

We implement a bidirectional Feature Pyramid Network (BiFPN) with the following improvements:

1. **Bidirectional feature flow** - Information flows both top-down and bottom-up
2. **Channel and spatial attention** - To focus on important features
3. **Scale-aware detection heads** - Specialized processing for different feature scales
4. **Weighted feature fusion** - Better integration of multi-scale features
5. **Increased input resolution** - 320x320 instead of 224x224 for better small plate detection

In [ ]:
# Create the enhanced BiFPN model
print("Creating enhanced BiFPN license plate detector...")
bifpn_model = create_bifpn_license_plate_detector(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Display model summary
bifpn_model.summary()

# Print model size information
trainable_count = np.sum([keras.backend.count_params(w) for w in bifpn_model.trainable_weights])
non_trainable_count = np.sum([keras.backend.count_params(w) for w in bifpn_model.non_trainable_weights])
print(f'Total parameters: {trainable_count + non_trainable_count:,}')
print(f'Trainable parameters: {trainable_count:,}')
print(f'Non-trainable parameters: {non_trainable_count:,}')

# Apply memory optimizations
print("Applying gradient checkpointing to reduce memory usage...")
bifpn_model = enable_gradient_checkpointing(bifpn_model)

## Step 5: GPU Memory Optimizations

In [ ]:
# Enable mixed precision training to reduce memory usage and increase speed on GPU
try:
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)
    print("Mixed precision policy set to", policy.name)
    print("Compute dtype:", policy.compute_dtype)
    print("Variable dtype:", policy.variable_dtype)
except Exception as e:
    print(f"Could not enable mixed precision: {e} - using default precision")

# Clean up memory before starting training
clean_memory()

# Optimize training data formats
X_train, X_val, y_train, y_val = optimize_memory_usage(X_train, y_train, X_val, y_val)

print("Memory optimization complete. Training should now be more stable on GPU.")

## Step 6: Advanced Training Configuration

In [ ]:
# Compile the model with our custom loss function
bifpn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=ultra_plate_detection_loss,  # Our enhanced loss function
    metrics=[
        enhanced_iou_metric,  # Primary metric for monitoring
        enhanced_ciou_loss,   # For monitoring size accuracy
        enhanced_focal_loss   # For monitoring hard example performance
    ]
)

# Set up callbacks with optimized hyperparameters
callbacks = [
    # Cosine annealing learning rate scheduler with warm restarts
    create_lr_scheduler(
        scheduler_type='cosine_restart',  
        initial_learning_rate=0.001,
        first_decay_steps=15,            # First cycle length
        t_mul=1.5,                       # Cycle length growth factor
        m_mul=0.75,                      # LR decay factor for each restart
        alpha=1e-5                       # Minimum LR value
    ),
    
    # More permissive early stopping focused on IoU improvement
    keras.callbacks.EarlyStopping(
        monitor='val_enhanced_iou_metric',
        patience=20,                     # Longer patience for better convergence
        restore_best_weights=True,
        mode='max'
    ),
    
    # Model checkpoint to save best model
    keras.callbacks.ModelCheckpoint(
        'enhanced_bifpn_license_plate_detector.h5',
        monitor='val_enhanced_iou_metric',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # TensorBoard logging
    keras.callbacks.TensorBoard(
        log_dir='./logs/bifpn_model',
        histogram_freq=1,
        update_freq='epoch'
    ),
    
    # Custom callback to reduce learning rate on plateau if needed
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_enhanced_iou_metric',
        factor=0.5,
        patience=10,
        min_lr=1e-6,
        mode='max',
        verbose=1
    )
]

# Create optimized data pipelines with improved batch size
BATCH_SIZE = 16  # Reduced from 24 to accommodate larger model and higher resolution

print("Creating efficient data pipeline...")
train_dataset, val_dataset = create_efficient_data_pipeline(
    X_train, y_train, X_val, y_val,
    batch_size=BATCH_SIZE
)

# Create on-the-fly augmentation function
tf_augmenter = create_tf_data_augmenter(cutout_prob=0.3, mixup_prob=0.2)

# Apply on-the-fly augmentation to the training dataset
augmented_train_dataset = train_dataset.map(
    tf_augmenter,
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

## Step 7: Model Training

In [ ]:
# Start training timer
start_time = time.time()

print("Starting Enhanced BiFPN model training...")
print("Training with the following improvements:")
print("- Bidirectional feature fusion through BiFPN")
print("- Enhanced attention mechanisms (channel, spatial, and dual)")
print("- Scale-aware detection heads for different feature levels")
print("- Ultra plate detection loss focusing on size accuracy")
print("- Small plate focused augmentation")
print("- Higher input resolution (320x320)")
print("- Cosine annealing LR schedule with warm restarts")

# Train the model
history = bifpn_model.fit(
    augmented_train_dataset,
    validation_data=val_dataset,
    epochs=70,  # Increased from 50 for better convergence
    callbacks=callbacks,
    verbose=1
)

# Calculate training time
training_time = time.time() - start_time
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)
print(f"Training completed in {hours}h {minutes}m {seconds}s")

# Reference to trained model
bifpn_trained_model = bifpn_model

# Save the final model
bifpn_model.save('bifpn_license_plate_detector_final.h5')

## Step 8: Training Visualization

In [ ]:
# Visualize training history
plt.figure(figsize=(15, 5))

# Plot training & validation loss
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.grid(alpha=0.3)

# Plot IoU metric
plt.subplot(1, 3, 2)
plt.plot(history.history['enhanced_iou_metric'])
plt.plot(history.history['val_enhanced_iou_metric'])
plt.title('IoU Metric')
plt.ylabel('IoU')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.grid(alpha=0.3)

# Plot focal loss component
plt.subplot(1, 3, 3)
plt.plot(history.history['enhanced_focal_loss'])
plt.plot(history.history['val_enhanced_focal_loss'])
plt.title('Focal Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Print final metrics
if history.history:
    epochs_trained = len(history.history['loss'])
    print(f"Total epochs trained: {epochs_trained}")
    
    print(f"Final training loss: {history.history['loss'][-1]:.4f}")
    print(f"Final validation loss: {history.history['val_loss'][-1]:.4f}")
    
    train_iou = history.history['enhanced_iou_metric'][-1]
    val_iou = history.history['val_enhanced_iou_metric'][-1]
    print(f"Final training IoU: {train_iou:.4f}")
    print(f"Final validation IoU: {val_iou:.4f}")
    
    # Get best IoU
    best_epoch = np.argmax(history.history['val_enhanced_iou_metric'])
    best_val_iou = history.history['val_enhanced_iou_metric'][best_epoch]
    print(f"Best validation IoU: {best_val_iou:.4f} (epoch {best_epoch+1})")
else:
    print("No training history found. Training may have failed.")

## Step 9: Model Evaluation

In [ ]:
# Generate predictions from the trained model
print("Generating predictions from the trained model...")
bifpn_predictions = bifpn_trained_model.predict(X_val)

# Run comprehensive evaluation
print("Running comprehensive evaluation...")
bifpn_results = evaluate_model_comprehensive(
    bifpn_trained_model,
    X_val,
    y_val,
    bifpn_predictions
)

print(f"BiFPN model mean IoU: {bifpn_results['mean_iou']:.4f}")
print(f"BiFPN model mAP@0.5: {bifpn_results['map50']:.4f}")

## Step 10: Error Analysis by Plate Size

In [ ]:
# Perform detailed error analysis by plate size categories
bifpn_error_analysis = analyze_error_patterns(
    model=bifpn_trained_model,
    X_val=X_val,
    y_val=y_val,
    y_pred=bifpn_predictions,
    plate_sizes=[box[2] * box[3] for box in y_val]
)

# Display detailed error metrics
print("\nDetailed Error Analysis:")
print(f"- Mean IoU: {bifpn_error_analysis['mean_iou']:.4f}")
print(f"- Position error: {bifpn_error_analysis['position_error']:.4f}")
print(f"- Size error: {bifpn_error_analysis['size_error']:.4f}")
print(f"- Aspect ratio error: {bifpn_error_analysis['aspect_ratio_error']:.4f}")

# Display error metrics by plate size category
print("\nPerformance by Plate Size:")
for size_cat, metrics in bifpn_error_analysis['by_size_category'].items():
    print(f"- {size_cat.capitalize()} plates ({metrics['count']} samples):")
    print(f"  - IoU: {metrics['iou']:.4f}")
    print(f"  - Position error: {metrics['position_error']:.4f}")
    print(f"  - Size error: {metrics['size_error']:.4f}")

## Step 11: Visualize Predictions

In [ ]:
# Visualize predictions on validation set
# Select specific examples from each plate size category
plate_areas = np.array([box[2] * box[3] for box in y_val])
small_indices = np.where(plate_areas < 0.05)[0]
medium_indices = np.where((plate_areas >= 0.05) & (plate_areas < 0.1))[0]
large_indices = np.where(plate_areas >= 0.1)[0]

# Sample from each category
num_samples = 3
small_samples = np.random.choice(small_indices, size=min(num_samples, len(small_indices)), replace=False)
medium_samples = np.random.choice(medium_indices, size=min(num_samples, len(medium_indices)), replace=False)
large_samples = np.random.choice(large_indices, size=min(num_samples, len(large_indices)), replace=False)

# Combine samples
sample_indices = np.concatenate([small_samples, medium_samples, large_samples])

# Create figure
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

# Plot predictions vs ground truth
for i, idx in enumerate(sample_indices):
    if i < len(axes):
        # Get ground truth and prediction
        true_box = y_val[idx]
        pred_box = bifpn_predictions[idx]
        
        # Calculate IoU for this sample
        iou = plot_predictions(X_val[idx], true_box, pred_box, ax=axes[i])
        
        # Add category label
        area = true_box[2] * true_box[3] * 100
        if area < 5:
            category = "Small"
        elif area < 10:
            category = "Medium"
        else:
            category = "Large"
            
        axes[i].set_title(f"{category} plate (Area: {area:.1f}%, IoU: {iou:.4f})")

plt.tight_layout()
plt.show()

## Step 12: Compare with Previous FPN Results

In [ ]:
# Load previous FPN evaluation results (if available)
try:
    # You could load saved metrics from a file
    # or use hardcoded values from previous runs
    prev_metrics = {
        'mean_iou': 0.2514,  # Previous mean IoU
        'small_iou': 0.2227,  # Previous IoU for small plates
        'medium_iou': 0.3247,  # Previous IoU for medium plates
        'large_iou': 0.4755,  # Previous IoU for large plates
        'size_error': 1.6132  # Previous size error
    }
    
    # Extract current metrics for comparison
    current_metrics = {
        'mean_iou': bifpn_error_analysis['mean_iou'],
        'small_iou': bifpn_error_analysis['by_size_category']['small']['iou'],
        'medium_iou': bifpn_error_analysis['by_size_category']['medium']['iou'],
        'large_iou': bifpn_error_analysis['by_size_category']['large']['iou'],
        'size_error': bifpn_error_analysis['size_error']
    }
    
    # Calculate improvement percentages
    improvements = {
        key: (current_metrics[key] - prev_metrics[key]) / prev_metrics[key] * 100 
        if key != 'size_error' else 
        (prev_metrics[key] - current_metrics[key]) / prev_metrics[key] * 100
        for key in prev_metrics
    }
    
    # Display comparison
    print("Comparison with Previous FPN Model:")
    print(f"Mean IoU: {prev_metrics['mean_iou']:.4f} → {current_metrics['mean_iou']:.4f} ({improvements['mean_iou']:+.1f}%)")
    print(f"Small plate IoU: {prev_metrics['small_iou']:.4f} → {current_metrics['small_iou']:.4f} ({improvements['small_iou']:+.1f}%)")
    print(f"Medium plate IoU: {prev_metrics['medium_iou']:.4f} → {current_metrics['medium_iou']:.4f} ({improvements['medium_iou']:+.1f}%)")
    print(f"Large plate IoU: {prev_metrics['large_iou']:.4f} → {current_metrics['large_iou']:.4f} ({improvements['large_iou']:+.1f}%)")
    print(f"Size error: {prev_metrics['size_error']:.4f} → {current_metrics['size_error']:.4f} ({improvements['size_error']:+.1f}%)")
    
    # Plot comparison bar chart
    categories = ['Mean IoU', 'Small Plate IoU', 'Medium Plate IoU', 'Large Plate IoU']
    prev_values = [prev_metrics['mean_iou'], prev_metrics['small_iou'], prev_metrics['medium_iou'], prev_metrics['large_iou']]
    curr_values = [current_metrics['mean_iou'], current_metrics['small_iou'], current_metrics['medium_iou'], current_metrics['large_iou']]
    
    x = np.arange(len(categories))
    width = 0.35
    
    plt.figure(figsize=(12, 6))
    plt.bar(x - width/2, prev_values, width, label='Previous FPN')
    plt.bar(x + width/2, curr_values, width, label='Enhanced BiFPN')
    
    plt.ylabel('IoU')
    plt.title('Performance Comparison')
    plt.xticks(x, categories)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for i, v in enumerate(prev_values):
        plt.text(i - width/2, v + 0.02, f"{v:.3f}", ha='center')
    
    for i, v in enumerate(curr_values):
        plt.text(i + width/2, v + 0.02, f"{v:.3f}", ha='center')
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Could not create comparison with previous model: {e}")
    print("Current model results:")
    print(f"- Mean IoU: {bifpn_error_analysis['mean_iou']:.4f}")
    for size_cat, metrics in bifpn_error_analysis['by_size_category'].items():
        print(f"- {size_cat.capitalize()} plate IoU: {metrics['iou']:.4f}")

## Step 13: Conclusion and Next Steps

In [ ]:
# Display final conclusion
print("\nEnhanced BiFPN License Plate Detection Results Summary:")
print(f"Overall mean IoU: {bifpn_error_analysis['mean_iou']:.4f}")

# Check if target IoU of 0.6 was achieved
target_iou = 0.6
if bifpn_error_analysis['mean_iou'] >= target_iou:
    print(f"✅ Successfully achieved target IoU of {target_iou}!")
    improvement = (bifpn_error_analysis['mean_iou'] - target_iou) / target_iou * 100
    print(f"   Performance is {improvement:.1f}% above the target.")
else:
    print(f"❌ Target IoU of {target_iou} not yet achieved.")
    shortfall = (target_iou - bifpn_error_analysis['mean_iou']) / target_iou * 100
    print(f"   Performance is {shortfall:.1f}% below the target.")
    
    # Suggestions for further improvement
    print("\nSuggestions for further improvement:")
    print("1. Increase model capacity with additional layers or filters")
    print("2. Further tune the loss function weights for specific error patterns")
    print("3. Increase image resolution to 384×384 or higher")
    print("4. Apply advanced learning techniques like knowledge distillation")
    print("5. Implement test-time augmentation for inference")

print("\nKey improvements in this implementation:")
print("1. Enhanced BiFPN architecture with bidirectional feature fusion")
print("2. Specialized loss function focusing on size estimation accuracy")
print("3. Targeted augmentation strategies for small plates")
print("4. Increased resolution (320×320) for better small object detection")
print("5. Advanced learning rate scheduling with cosine annealing")

print("\nNext steps for deployment:")
print("1. Model export to TFLite or ONNX format for production")
print("2. Implementation of a post-processing pipeline for multi-plate scenarios")
print("3. Integration with license plate recognition (LPR) system")
print("4. Performance optimization for real-time inference")